In [1]:
import pandas as pd
import numpy as np

1. Chuẩn bị dữ liệu

In [27]:
item =  pd.read_csv("D:/Học tập/Học kì 20231/Hệ hĩnh trợ quyết độ/DMC_2017_task/items.csv", sep = "|")
item.drop_duplicates(keep='first', inplace=True)
item['pharmForm'] = item['pharmForm'].str.upper()

train  = pd.read_csv("D:/Học tập/Học kì 20231/Hệ hĩnh trợ quyết độ/DMC_2017_task/train.csv", sep = "|")
train = train.merge(item[['pid', 'rrp']], on= 'pid')

In [86]:
train.fillna(0, inplace=True)
int_columns = train.select_dtypes(include=['int64']).columns
train[int_columns] = train[int_columns].astype('int')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756003 entries, 0 to 2756002
Data columns (total 12 columns):
 #   Column           Dtype  
---  ------           -----  
 0   lineID           int32  
 1   day              int32  
 2   pid              int32  
 3   adFlag           int32  
 4   availability     int32  
 5   competitorPrice  float64
 6   click            int32  
 7   basket           int32  
 8   order            int32  
 9   price            float64
 10  revenue          float64
 11  rrp              float64
dtypes: float64(4), int32(8)
memory usage: 168.2 MB


In [87]:
Multi_TSR = {}
cache_day = {}
numof_pid={}
pid = item['pid'].unique().tolist()
for i in range(len(pid)):
    Multi_TSR[pid[i]] = {}
    numof_pid[pid[i]] = 0
    cache_day[pid[i]] = 0
for i in range(len(train)):
    cache_pid = int(train.loc[i, 'pid'])
    tmp_day = train.loc[i, 'day']
    if tmp_day > cache_day[cache_pid]:
        cache_day[cache_pid] = tmp_day
        numof_pid[cache_pid] += 1
        Multi_TSR[cache_pid].update({tmp_day: {'pid': cache_pid, 'day': tmp_day,'rrp': train.loc[i, 'rrp'], 'competitorPrice': train.loc[i, 'competitorPrice'], 'click': train.loc[i, 'click'], 'basket': train.loc[i, 'basket'], 'order': train.loc[i, 'order'], 'price': train.loc[i,'price']}})
    elif tmp_day == cache_day[cache_pid]:
        Multi_TSR[cache_pid][tmp_day]['click'] += train.loc[i, 'click']
        Multi_TSR[cache_pid][tmp_day]['basket'] += train.loc[i, 'basket']
        Multi_TSR[cache_pid][tmp_day]['order'] += train.loc[i, 'order']
        

In [88]:
del cache_day
print(len(Multi_TSR))
for i in range(len(pid)):
    if numof_pid[pid[i]] < 2:
        Multi_TSR.pop(pid[i])
print(len(Multi_TSR))

22035
21453


In [89]:
Multi_TSR[12002]

{7: {'pid': 12002,
  'day': 7,
  'rrp': 26.95,
  'competitorPrice': 21.1,
  'click': 0,
  'basket': 1,
  'order': 1,
  'price': 18.85},
 11: {'pid': 12002,
  'day': 11,
  'rrp': 26.95,
  'competitorPrice': 19.79,
  'click': 0,
  'basket': 1,
  'order': 0,
  'price': 20.37},
 13: {'pid': 12002,
  'day': 13,
  'rrp': 26.95,
  'competitorPrice': 19.79,
  'click': 0,
  'basket': 1,
  'order': 0,
  'price': 21.09},
 17: {'pid': 12002,
  'day': 17,
  'rrp': 26.95,
  'competitorPrice': 19.79,
  'click': 1,
  'basket': 0,
  'order': 0,
  'price': 24.93},
 18: {'pid': 12002,
  'day': 18,
  'rrp': 26.95,
  'competitorPrice': 19.79,
  'click': 0,
  'basket': 1,
  'order': 0,
  'price': 24.93},
 21: {'pid': 12002,
  'day': 21,
  'rrp': 26.95,
  'competitorPrice': 22.32,
  'click': 1,
  'basket': 1,
  'order': 0,
  'price': 21.29},
 22: {'pid': 12002,
  'day': 22,
  'rrp': 26.95,
  'competitorPrice': 21.1,
  'click': 0,
  'basket': 1,
  'order': 1,
  'price': 21.71},
 24: {'pid': 12002,
  'day': 24

In [90]:
data = {}
count = 0
for _, value in Multi_TSR.items():
    sub_key_old = 0
    sub_value_old = []
    for sub_key, sub_value in value.items():
        if sub_key_old == 0:
            sub_key_old = sub_key
            sub_value_old.append(sub_value)
        elif sub_key_old == (sub_key - 1):
            sub_key_old = sub_key
            sub_value_old.append(sub_value)
        else:
            if len(sub_value_old)==1:
                sub_key_old = sub_key
                sub_value_old = [sub_value]
            elif len(sub_value_old) > 1:
                data.update({count: sub_value_old})
                sub_value_old = [sub_value]
                sub_key_old = sub_key
                count = count + 1

In [91]:
len(data)

102020

In [94]:
import json

class npEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.int32):
            return int(obj)
        return json.JSONEncoder.default(self, obj)

file_path = 'output.json'  # Đường dẫn tới file
with open(file_path, 'w') as file:
    json.dump(data, file, cls= npEncoder)

print(f"Đã ghi dữ liệu vào file: '{file_path}'")

Đã ghi dữ liệu vào file: 'output.json'
